In [ ]:
from backtester import BackTester
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
plt.rcParams["figure.figsize"] = (15,10)
sns.set_theme()

In [ ]:
def evaluate_benchmark_for_day(day):
    ret = backtest.benchmark_changes.loc[day][0]
    if math.isnan(ret):
        return 0
    else:
        return ret

def equity_curve_benchmark(starting_capital,leverage,benchmark_returns):
    equity = [starting_capital]
    for i in range(len(benchmark_returns)):
        equity.append(equity[-1]*(1 + benchmark_returns[i]*leverage))
    return equity


In [ ]:
backtest = BackTester(file_name = 'combined_minute.csv',
                     start_date='2011-05-02',
                     end_date ='2022-02-18',
                     gex_bins = 16,
                     starting_capital = 1000000,
                     leverage = 1.2)

In [ ]:
backtest.data

In [ ]:
backtest.benchmark_changes = backtest.data.groupby(level =0).nth(-1)[['close_ask_price']].pct_change()
rets = [evaluate_benchmark_for_day(day) for day in backtest.data.index.unique()]

In [ ]:
backtest.evaluate_strategy(mr_sma = 20,
                     mr_threshold = 0.001,
                     mom_period = 30)

In [ ]:
df = pd.DataFrame(backtest.equity_curve_strategy[1:], index = backtest.data.index.unique()).rename({0:'Strategy'}, axis = 1)
df['SPY'] = equity_curve_benchmark(1000000, 1.2, rets)[1:]
df.index = pd.to_datetime(df.index)

In [ ]:
plt.plot(df['Strategy'])
plt.plot(df['SPY'])
plt.xlabel('Date')
plt.ylabel('Strategy Equity')
plt.title('Intraday Gamma Weighted Mean Reversion & Momentum Strategy vs SPY Buy and Hold')